In [58]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt

base_path = "/kaggle/input/trends-assessment-prediction/"

In [59]:
# read non .mat data into dataframes
fnc = pd.read_csv(base_path + "fnc.csv")
loading = pd.read_csv(base_path + "loading.csv")
train_scores = pd.read_csv(base_path + "train_scores.csv")

In [60]:
# basic preproccessing

data_list = [loading, train_scores]

for data in data_list:
    for col in data.columns:
        mean = np.mean(data[col])
        data[col].fillna(mean)
        

In [61]:
# splitting into train and pred without .mat files
loading_train_test = loading[loading.Id.isin(train_scores.Id)]
loading_for_pred = loading[~loading.Id.isin(train_scores.Id)]
print(loading_train_test.shape)
print(loading_for_pred.shape)

# fnc_train_test = fnc[fnc.Id.isin(train_scores.Id)]
# fnc_for_pred = fnc[~fnc.Id.isin(train_scores.Id)]
# print(fnc_train.shape)
# print(fnc_test.shape)

(5877, 27)
(5877, 27)


In [81]:
# splitting data into further train and test
# can be improved by training several models on several folds eg. Stratified shuffle split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(loading_train_test, train_scores, test_size=0.2,random_state=42)

data_list = [X_train, X_test, y_train, y_test]

for data in data_list:
    for col in data.columns:
        mean = np.mean(data[col])
        data[col].fillna(mean)
        
# first row coltains int64 which cannot be handled by regressor

X_train = X_train.reset_index()
y_train = y_train.reset_index()
X_train = X_train.drop(columns=['index', 'Id']).replace([np.inf, -np.inf], 0)
y_train = y_train.drop(columns=['index','Id']).replace([np.inf, -np.inf], 0)

In [82]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701 entries, 0 to 4700
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           4701 non-null   float64
 1   domain1_var1  4339 non-null   float64
 2   domain1_var2  4339 non-null   float64
 3   domain2_var1  4667 non-null   float64
 4   domain2_var2  4667 non-null   float64
dtypes: float64(5)
memory usage: 183.8 KB


In [72]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701 entries, 0 to 4700
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   IC_01   4701 non-null   float64
 1   IC_07   4701 non-null   float64
 2   IC_05   4701 non-null   float64
 3   IC_16   4701 non-null   float64
 4   IC_26   4701 non-null   float64
 5   IC_06   4701 non-null   float64
 6   IC_10   4701 non-null   float64
 7   IC_09   4701 non-null   float64
 8   IC_18   4701 non-null   float64
 9   IC_04   4701 non-null   float64
 10  IC_12   4701 non-null   float64
 11  IC_24   4701 non-null   float64
 12  IC_15   4701 non-null   float64
 13  IC_13   4701 non-null   float64
 14  IC_17   4701 non-null   float64
 15  IC_02   4701 non-null   float64
 16  IC_08   4701 non-null   float64
 17  IC_03   4701 non-null   float64
 18  IC_21   4701 non-null   float64
 19  IC_28   4701 non-null   float64
 20  IC_11   4701 non-null   float64
 21  IC_20   4701 non-null   float64
 22  

In [64]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

features = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')

model = RandomForestRegressor(
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5
)
cv = KFold(n_splits = 5, shuffle=True, random_state=29)
grid = {'n_estimators':[5,10,20,100]}
gs = GridSearchCV(model, grid, n_jobs=-1, cv=cv, verbose=1, scoring='neg_mean_absolute_error')

In [83]:

best_models = {}
for col in features:
    gs.fit(X_train, y_train[col])
    best_models[col] = gs.best_estimator_
    print(gs.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   16.3s finished


-8.104662715604578
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').